<a href="https://colab.research.google.com/github/jasminl/adaptics/blob/master/onnx_create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install onnx
!pip install onnxruntime
!pip install onnxruntime_extensions
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.2 MB/s eta 0:00:00


In [1]:
from onnx import helper, onnx_pb as onnx_proto, save_model
from onnxruntime_extensions import onnx_op, PyCustomOpDef, make_onnx_model

nodes = [helper.make_node('CustomOp', ['input'], ['output'], domain='ai.onnx.contrib')]

@onnx_op(op_type="CustomOp",outputs=[PyCustomOpDef.dt_float])
def custom_two_op(f):
    print('Arbitrary code')
    return f

input = helper.make_tensor_value_info('input', onnx_proto.TensorProto.FLOAT, [1, 1])
output = helper.make_tensor_value_info('output', onnx_proto.TensorProto.FLOAT, [1, 1])

graph = helper.make_graph(nodes, 'test', [input], [output])
model = helper.make_model(graph,
                          opset_imports=[helper.make_operatorsetid('ai.onnx.contrib', 1)],
                          ir_version=7)
save_model(model, 'mymodel.onnx')


In [1]:
from onnx import load_model
import onnxruntime as _ort
from onnxruntime_extensions import (
    onnx_op, PyCustomOpDef, make_onnx_model,
    get_library_path as _get_library_path)
import numpy as np

so = _ort.SessionOptions()

# Define a custom operator: this must be defined before the call to register_custom_ops_library
@onnx_op(op_type="CustomOp",outputs=[PyCustomOpDef.dt_float])
def custom_two_op(f):
    print('Arbitrary code')
    return np.round(f)

so.register_custom_ops_library(_get_library_path())  # Must first call any @onnx_op before this

new_model = load_model('mymodel.onnx')

sess = _ort.InferenceSession(new_model.SerializeToString(), so)
res = sess.run(None, {'input': np.random.rand(1, 1).astype(np.float32)})

Arbitrary code
[array([[1.]], dtype=float32)]


In [15]:
import json
import io
import onnx
from onnx import load_model
new_model = load_model('mymodel.onnx')

# Do model checks (if greater than 2GB, use file path instead: https://github.com/onnx/onnx/blob/main/docs/PythonAPIOverview.md)
try:
    onnx.checker.check_model(new_model)
except onnx.checker.ValidationError as e:
    print(f"The model is invalid: {e}")
else:
    print("The model is valid!")

# Check for presence of custom operators (even if implementation is lacking)

print(new_model)
for node in new_model.graph.node:
    if node.domain != '' and node.domain != 'ai.onnx':
        print('Custom node found')
        print(node)
for node in new_model.graph.input:
    print(node)

The model is valid!
ir_version: 7
graph {
  node {
    input: "input"
    output: "output"
    op_type: "CustomOp"
    domain: "ai.onnx.contrib"
  }
  name: "test"
  input {
    name: "input"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 1
          }
          dim {
            dim_value: 1
          }
        }
      }
    }
  }
  output {
    name: "output"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 1
          }
          dim {
            dim_value: 1
          }
        }
      }
    }
  }
}
opset_import {
  domain: "ai.onnx.contrib"
  version: 1
}

Custom node found
input: "input"
output: "output"
op_type: "CustomOp"
domain: "ai.onnx.contrib"

name: "input"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 1
      }
    }
  }
}

